In [82]:
import pandas as pd
import numpy as np
import math
import pickle

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

import matlab.engine as engi
import matlab as mat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from pyearth import Earth

from src import SMOTE
from src import CFS
from src import metrices_V2 as metrices

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

import matplotlib.pyplot as plt

In [83]:
def load_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    commit_guru_path = 'data/commit_guru/' + project + '.csv'
    commit_guru_df = pd.read_csv(commit_guru_path)
#     print(commit_guru_df.columns)
    commit_guru_df = commit_guru_df.drop(labels = ['parent_hashes','author_name','author_name',
                                                   'author_email','author_date','fileschanged',
                                                   'author_date_unix_timestamp', 'commit_message','commit_hash',
                                                  'classification', 'fix','fixes',],axis=1)

    df = commit_guru_df
    df.rename(columns={"contains_bug": "Bugs"},inplace=True)
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    y.fillna(False,inplace=True)
    df['Bugs'] = y
    df.dropna(inplace=True)
    cols = df.columns.tolist()
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    return X,y

def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

In [103]:
def run_self(project):
    X,y = load_data(project)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    loc = X_test['la'] + X_test['lt']
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = GaussianNB()
    clf.fit(X_train,y_train)
    predicted = clf.predict(X_test)
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa

In [93]:
def run_self_CFS(project):
    X,y = load_data(project)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    loc = X_test['la'] + X_test['lt']
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    df_smote,cols = apply_cfs(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = RandomForestClassifier()
    clf.fit(X_train,y_train)
    predicted = clf.predict(X_test[cols[:-1]])
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa

In [94]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()

In [104]:
precision_list = {}
recall_list = {}
pf_list = {}
f1_list = {}
g_list = {}
auc_list = {}
pci_20_list = {}
ifa_list = {}
for project in projects:
    try:
        if project == '.DS_Store':
            continue
    #     if project != 'guice':
    #         continue
        print("+++++++++++++++++   "  + project + "  +++++++++++++++++")
        recall,precision,pf,f1,g_score,auc,pci_20,ifa = run_self(project)
        recall_list[project] = recall
        precision_list[project] = precision
        pf_list[project] = pf
        f1_list[project] = f1
        g_list[project] = g_score
        auc_list[project] = auc
        pci_20_list[project] = pci_20
        ifa_list[project] = ifa
    except Exception as e:
        print(e)
        continue
final_result = {}
final_result['precision'] = precision_list
final_result['recall'] = recall_list
final_result['pf'] = pf_list
final_result['f1'] = f1_list
final_result['g'] = g_list
final_result['auc'] = auc_list
final_result['pci_20'] = pci_20_list
final_result['ifa'] = ifa_list
with open('results/Performance/process_1000_nb.pkl', 'wb') as handle:
    pickle.dump(final_result, handle, protocol=pickle.HIGHEST_PROTOCOL)

+++++++++++++++++   org.alloytools.alloy  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.97      1.00      0.99        38
        True       0.00      0.00      0.00         1

    accuracy                           0.97        39
   macro avg       0.49      0.50      0.49        39
weighted avg       0.95      0.97      0.96        39

+++++++++++++++++   qpython  +++++++++++++++++
index 1 is out of bounds for axis 0 with size 1
+++++++++++++++++   friendlychat-android  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.92      1.00      0.96        11
        True       0.00      0.00      0.00         1

    accuracy                           0.92        12
   macro avg       0.46      0.50      0.48        12
weighted avg       0.84      0.92      0.88        12

+++++++++++++++++   paho.mqtt.android  +++++++++++++++++
              precision    recall  f1-score   support

       False       

index 1 is out of bounds for axis 0 with size 1
+++++++++++++++++   CircleSeekbar  +++++++++++++++++
index 1 is out of bounds for axis 0 with size 1
+++++++++++++++++   TaskScheduler  +++++++++++++++++
index 1 is out of bounds for axis 0 with size 1
+++++++++++++++++   TextLayoutBuilder  +++++++++++++++++
index 1 is out of bounds for axis 0 with size 1
+++++++++++++++++   BubbleView  +++++++++++++++++
              precision    recall  f1-score   support

       False       1.00      0.97      0.99        40
        True       0.00      0.00      0.00         0

    accuracy                           0.97        40
   macro avg       0.50      0.49      0.49        40
weighted avg       1.00      0.97      0.99        40

+++++++++++++++++   Mappedbus  +++++++++++++++++
index 1 is out of bounds for axis 0 with size 1
+++++++++++++++++   project-rome  +++++++++++++++++
index 1 is out of bounds for axis 0 with size 1
+++++++++++++++++   sbt-android  +++++++++++++++++
index 1 is out of bo

              precision    recall  f1-score   support

       False       0.99      0.96      0.98       183
        True       0.12      0.50      0.20         2

    accuracy                           0.96       185
   macro avg       0.56      0.73      0.59       185
weighted avg       0.98      0.96      0.97       185

+++++++++++++++++   ToggleButton  +++++++++++++++++
              precision    recall  f1-score   support

       False       1.00      0.67      0.80        18
        True       0.00      0.00      0.00         0

    accuracy                           0.67        18
   macro avg       0.50      0.33      0.40        18
weighted avg       1.00      0.67      0.80        18

+++++++++++++++++   2048-android  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.89      0.57      0.69        30
        True       0.00      0.00      0.00         2

    accuracy                           0.53        32
   macro avg       0.45 

              precision    recall  f1-score   support

       False       1.00      0.90      0.95       242
        True       0.04      1.00      0.07         1

    accuracy                           0.90       243
   macro avg       0.52      0.95      0.51       243
weighted avg       1.00      0.90      0.94       243

+++++++++++++++++   alicloud-android-demo  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.97      0.86      0.91        65
        True       0.10      0.33      0.15         3

    accuracy                           0.84        68
   macro avg       0.53      0.60      0.53        68
weighted avg       0.93      0.84      0.88        68

+++++++++++++++++   hugo  +++++++++++++++++
              precision    recall  f1-score   support

       False       1.00      0.58      0.74        36
        True       0.12      1.00      0.21         2

    accuracy                           0.61        38
   macro avg       0.56

              precision    recall  f1-score   support

       False       0.94      0.87      0.90        38
        True       0.17      0.33      0.22         3

    accuracy                           0.83        41
   macro avg       0.55      0.60      0.56        41
weighted avg       0.89      0.83      0.85        41

+++++++++++++++++   ColorPickerView  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.97      0.64      0.77        55
        True       0.09      0.67      0.16         3

    accuracy                           0.64        58
   macro avg       0.53      0.65      0.46        58
weighted avg       0.93      0.64      0.74        58

+++++++++++++++++   GroupedRecyclerViewAdapter  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.88      0.71      0.79        21
        True       0.14      0.33      0.20         3

    accuracy                           0.67        24
   macr

...

              precision    recall  f1-score   support

       False       0.88      0.96      0.92       687
        True       0.42      0.19      0.26       112

    accuracy                           0.85       799
   macro avg       0.65      0.57      0.59       799
weighted avg       0.81      0.85      0.82       799

+++++++++++++++++   handlebars.java  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.98      0.37      0.54       236
        True       0.38      0.98      0.55        94

    accuracy                           0.55       330
   macro avg       0.68      0.68      0.55       330
weighted avg       0.81      0.55      0.54       330

+++++++++++++++++   stringtemplate4  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.78      0.93      0.85       239
        True       0.74      0.42      0.54       109

    accuracy                           0.77       348
   macro avg      

              precision    recall  f1-score   support

       False       0.89      0.90      0.90       695
        True       0.38      0.35      0.36       118

    accuracy                           0.82       813
   macro avg       0.64      0.63      0.63       813
weighted avg       0.82      0.82      0.82       813

+++++++++++++++++   PageTurner  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.85      0.64      0.73       327
        True       0.38      0.67      0.48       108

    accuracy                           0.65       435
   macro avg       0.62      0.65      0.61       435
weighted avg       0.74      0.65      0.67       435

+++++++++++++++++   heroic  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.75      0.94      0.84       295
        True       0.44      0.13      0.20       105

    accuracy                           0.73       400
   macro avg       0.60      0.5

              precision    recall  f1-score   support

       False       0.92      0.22      0.36       486
        True       0.22      0.91      0.35       116

    accuracy                           0.36       602
   macro avg       0.57      0.57      0.36       602
weighted avg       0.78      0.36      0.36       602

+++++++++++++++++   pebble  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.74      0.96      0.84       277
        True       0.73      0.24      0.37       123

    accuracy                           0.74       400
   macro avg       0.74      0.60      0.60       400
weighted avg       0.74      0.74      0.69       400

+++++++++++++++++   CameraView  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.69      0.62      0.65        82
        True       0.78      0.83      0.81       136

    accuracy                           0.75       218
   macro avg       0.74      0.7

              precision    recall  f1-score   support

       False       0.83      0.66      0.74       327
        True       0.49      0.71      0.58       149

    accuracy                           0.67       476
   macro avg       0.66      0.68      0.66       476
weighted avg       0.72      0.67      0.69       476

+++++++++++++++++   opsu  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.89      0.13      0.23       182
        True       0.50      0.98      0.66       158

    accuracy                           0.53       340
   macro avg       0.69      0.56      0.44       340
weighted avg       0.71      0.53      0.43       340

+++++++++++++++++   metrics  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.83      0.92      0.87       714
        True       0.35      0.18      0.24       167

    accuracy                           0.78       881
   macro avg       0.59      0.55    

              precision    recall  f1-score   support

       False       0.78      0.97      0.86      1140
        True       0.46      0.10      0.16       346

    accuracy                           0.76      1486
   macro avg       0.62      0.53      0.51      1486
weighted avg       0.71      0.76      0.70      1486

+++++++++++++++++   nokogiri  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.79      0.93      0.85      1255
        True       0.44      0.18      0.25       385

    accuracy                           0.75      1640
   macro avg       0.61      0.55      0.55      1640
weighted avg       0.70      0.75      0.71      1640

+++++++++++++++++   rapidoid  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.82      0.92      0.87      1686
        True       0.23      0.10      0.14       374

    accuracy                           0.77      2060
   macro avg       0.53      0.5

In [100]:
for key in final_result.keys():
    print(len(final_result[key].values()))
    print(key,np.nanmedian(list(final_result[key].values())))

976
precision 0.36
976
recall 0.68
976
pf 0.24
976
f1 0.47
976
g 0.69
976
auc 0.7111666666666667
976
pci_20 0.78
976
ifa 4.0


In [ ]:
_precision = list(precision_list.values())
_recall = list(recall_list.values())
_pf = list(pf_list.values())
_f1 = list(f1_list.values())
_g = list(g_list.values())
_auc = list(auc_list.values())
print(np.median(_precision),np.median(_recall),np.median(_pf),np.median(_f1),np.median(_g),np.median(_auc))
fig = plt.figure(num=None, figsize = (20,4), facecolor='w', edgecolor='k')
ax = fig.add_subplot(161)
ax.boxplot(_precision)
ax.set_title('Precision',size = 15)
ax = fig.add_subplot(162)
ax.boxplot(_recall)
ax.set_title('Recall',size = 15)
ax = fig.add_subplot(163)
ax.boxplot(_pf)
ax.set_title('pf',size = 15)
ax = fig.add_subplot(164)
ax.boxplot(_f1)
ax.set_title('f1',size = 15)
ax = fig.add_subplot(165)
ax.boxplot(_g)
ax.set_title('g',size = 15)
ax = fig.add_subplot(166)
ax.boxplot(_auc)
ax.set_title('auc',size = 15)
# fig.savefig('without_process.png')